In [51]:
%load_ext autoreload
%autoreload 2

from copy import deepcopy
from typing import List
import numpy as np
import sklearn
from celer import Lasso
import random
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.base import BaseEstimator, RegressorMixin, ClassifierMixin
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
from scipy.linalg import hadamard
from synthetic_combinations import *
from sklearn import preprocessing
from fancyimpute import *
from horizontal_matrix_completion import *
from scipy.sparse import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generate Test Data  

## Define Parameters

In [47]:
np.random.seed(42)
N = 50
p = 10
r  = 3
s = int(p**(1.5))


## Generate Alpha Matrix and Potential Outcome Matrix

In [48]:
# Cell for generating alpha matrix
rvs = stats.norm().rvs
rank_users = []
for i in range(r):
    alpha_u = random(2**p,1,density=(s)/(2**p),data_rvs = rvs).toarray()
    rank_users.append(alpha_u[:,0])
rank_users = np.array(rank_users)
B = np.random.dirichlet([1]*r, N - r)
remaining_users = np.matmul(B,rank_users)
all_users = np.concatenate((rank_users,remaining_users))
all_users = normalize(all_users, axis=1, norm='l2')

TypeError: 'module' object is not callable

In [49]:
# Cell for generating potential outcome matrix 
fourier_characteristic_matrix = hadamard(2**p)
potential_outcome_matrix = np.matmul(all_users,fourier_characteristic_matrix)
#min_max_scaler = preprocessing.MinMaxScaler(feature_range = (-1,1))
#potential_outcome_matrix = min_max_scaler.fit_transform(potential_outcome_matrix)

## Generate Observation Pattern and Observation Matrix from Uniform Sampling

In [5]:
#Generate indices for donor units 
from random import choices
num_donor_units = 2*r
num_donor_unit_observations = s*p
num_non_donor_unit_observations = r**4
donor_unit_observations = choices(range(2**p), k = num_donor_unit_observations)
non_donor_unit_observations = choices(range(2**p), k = num_non_donor_unit_observations)

In [6]:
observation_matrix = np.empty((N,2**p,))
observation_matrix[:] = np.nan

In [7]:
observation_matrix[:num_donor_units,donor_unit_observations] = potential_outcome_matrix[:num_donor_units,donor_unit_observations]
observation_matrix[num_donor_units:,non_donor_unit_observations] =  potential_outcome_matrix[num_donor_units:,non_donor_unit_observations]
#donor_unit_observation_matrix = 
observation_matrix.shape        

(50, 1024)

# Horizontal Regression

In [45]:
horizontal_matrix = horizontal_matrix_completion()
lasso_regression_observation_matrix = horizontal_matrix.fit(deepcopy(observation_matrix))

100%|████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 17.09it/s]


In [46]:
mean_squared_error(potential_outcome_matrix[:,:],lasso_regression_observation_matrix[:,:])

0.7733262353776496

# Matrix Completion 

In [50]:
#X_filled_nnm = NuclearNormMinimization().fit_transform(observation_matrix)
#X_incomplete_normalized = BiScaler().fit_transform(observation_matrix)
#X_matrix_completion = SoftImpute().fit_transform(observation_matrix)
#X_filled_knn = KNN(k=3).fit_transform(observation_matrix)
X_matrix_completion = MatrixFactorization(rank = r).fit_transform(deepcopy(observation_matrix))

[MatrixFactorization] Iter 10: observed MAE=0.439623 rank=3
[MatrixFactorization] Iter 20: observed MAE=0.346861 rank=3
[MatrixFactorization] Iter 30: observed MAE=0.249797 rank=3
[MatrixFactorization] Iter 40: observed MAE=0.141156 rank=3
[MatrixFactorization] Iter 50: observed MAE=0.105902 rank=3


In [73]:
mean_squared_error(potential_outcome_matrix,X_matrix_completion)

0.9045627705655551

# Synthetic Combinations 

In [35]:
sc = synth_combo()

## Do Horizontal Regression 

In [36]:
donor_unit_indices = [i for i in range(num_donor_units)]
horizontal_regression_observation_matrix = sc.horizontal_fit(deepcopy(observation_matrix),donor_unit_indices)

100%|██████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  9.43it/s]


In [37]:
donor_unit_imputed_matrix = horizontal_regression_observation_matrix[donor_unit_indices,:]
mean_squared_error(donor_unit_imputed_matrix,potential_outcome_matrix[:num_donor_units,:])

0.006985592503999044

## Vertical Regression 

In [38]:
vertical_regression_observation_matrix = sc.vertical_fit(deepcopy(horizontal_regression_observation_matrix),donor_unit_indices,use_cv=False)

100%|██████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 5912.96it/s]


In [39]:
mean_squared_error(vertical_regression_observation_matrix,potential_outcome_matrix)

0.4239909709372751

In [24]:
cv_error = np.empty((3,3))
cv_error[:] = np.nan
#cv_error